### DEE2 STAR counts
full processing from dee2 to h5 gse x gene file. symbol in genes. gene_sybol in genes. 

In [1]:
%%time
import datetime
import bz2
import h5py
import numpy as np
import pandas as pd
import scipy.sparse as sps
from scipy.sparse import isspmatrix_csr,csr_matrix,csc_matrix,issparse,isspmatrix_csr
import math
cache_dir="G://datasets/DEE2/"
#cache_dir="./"
organism_name= 'ecoli'#'hsapiens' 'ecoli' 'mmusculus' 'athaliana' 'drerio' 'scerevisiae'
metadata_file = cache_dir+"metadata/"+ organism_name+"_metadata_20210225.tsv"
data_file_name= cache_dir+"raw_files/star_counts/"+organism_name+"_se_20210224.tsv.bz2" #celegans_se_20200530.tsv.bz2 dmelanogaster_se_20200530.tsv.bz2 ecoli_se_20200530.tsv.bz2
#drerio_se_20200530.tsv.bz2
#hsapiens_se_20200528.tsv.bz2
h5_gsm_name=cache_dir+"star_h5/"+organism_name+ "_star_matrix_20210224.h5"
#h5_gsm_name = "/scratch2/mkleverov/dee2/star_h5/"+organism_name+ "_star_matrix.h5"
gene_info_file_name =cache_dir+"GeneInfo/"+organism_name+"_GeneInfo.tsv"
dt = h5py.special_dtype(vlen=np.str)
now = datetime.datetime.now()
print("make gene index...")
gene_info=pd.read_csv(filepath_or_buffer = gene_info_file_name, sep="\t")
gene_ind= {gene:ind for ind,gene in enumerate(gene_info["GeneID"])}
n_genes=len(gene_ind)
print("gene quantity: "+str(n_genes))
print("read meta file...")
iter_meta = pd.read_csv(filepath_or_buffer=metadata_file,sep="\t", iterator=True, chunksize=1000)
meta_df = pd.concat([chunk for chunk in iter_meta])
geo_acc_name = meta_df.columns[5]#'GSE_accession' # may be "GSE_accession" or "Sample_name"
geo_gse_name = meta_df.columns[6]#'Sample_name' # may be "GEO_series" "SampleName" or "Sample_name"(ecoli)
meta_df[geo_gse_name]= meta_df[geo_gse_name].astype(str)
true_srr=len(meta_df)
print("real SRR quantity: " +str(true_srr))
meta_df=meta_df[meta_df[geo_acc_name].str.startswith('GSM')]
n_srr= len(meta_df)
print("SRRs with gse: "+str(n_srr))
print("make SRR and gsm indexes...")
srr_to_gsm= dict(zip(meta_df["SRR_accession"],meta_df[geo_acc_name]))
srr_ind= {srr:ind for ind,srr in enumerate(srr_to_gsm.keys())}
gsm_ind= {gsm:ind for ind,gsm in enumerate(pd.unique(meta_df[geo_acc_name]))}
n_gsm= len(gsm_ind)
print("GSMs quantity:" )
gsm_from_srr_matrix= sps.lil_matrix((n_gsm,n_srr))
for srr_key in srr_to_gsm:
    gsm_from_srr_matrix[gsm_ind[srr_to_gsm[srr_key]],srr_ind[srr_key]]=1
print("start grouping...")
col_list = [geo_acc_name,"SRR_accession","Model","LoadDate","LibrarySelection","LibrarySource","LibraryStrategy","ScientificName",geo_gse_name,"Consent","ReleaseDate","QC_summary","BioSample"]
meta_df=meta_df[col_list]
meta_df =meta_df.groupby(geo_acc_name).agg([lambda x: ';'.join(x)])
print("groupped")
meta_df=meta_df.loc[gsm_ind.keys()]
print("reordered")
print("start h5...")
with h5py.File(h5_gsm_name, 'w') as h5_gse:
    print("create file and write meta...")
    h5_gse.create_dataset('/meta/info/version', data="9")
    h5_gse.create_dataset('/meta/info/creation_date', data=now.strftime("%Y.%m.%d"))
    dt = h5py.special_dtype(vlen=np.str)
    h5_gse.create_dataset('/meta/genes/ensembl_gene_id',data=np.array(list(gene_ind.keys()),dtype=dt),chunks =(len(gene_ind),),compression="gzip", compression_opts=7)
    h5_gse.create_dataset('/meta/genes/genes',data=np.array(gene_info["GeneSymbol"],dtype=dt),chunks =(len(gene_ind),),compression="gzip", compression_opts=7)
    h5_gse.create_dataset('/meta/genes/gene_symbol',data=np.array(gene_info["GeneSymbol"],dtype=dt),chunks =(len(gene_ind),),compression="gzip", compression_opts=7)
    h5_gse.create_dataset('/meta/info/description', data="Contain all SRRs from dee2 for which sample_geo_accession starts with gse. Star estimates was groupped by sample_geo_accession and summed up")
    h5_gse.create_dataset('/meta/samples/geo_accession',data=np.array(list(gsm_ind.keys()), dtype =dt),chunks =(2000,),compression="gzip", compression_opts=5)
    h5_gse.create_dataset('/meta/samples/SRR_accession',data= np.reshape(np.array(meta_df["SRR_accession"]),-1),dtype=dt,chunks =(2000,),compression="gzip", compression_opts=5)
    h5_gse.create_dataset('/meta/samples/instrument_model',data=np.reshape(np.array(meta_df["Model"]),-1),dtype=dt,chunks =(2000,),compression="gzip", compression_opts=5)
    h5_gse.create_dataset('/meta/samples/last_update_date',data=np.reshape(np.array(meta_df["LoadDate"]),-1),dtype=dt,chunks =(2000,),compression="gzip", compression_opts=5)
    h5_gse.create_dataset('/meta/samples/library_selection',data=np.reshape(np.array(meta_df["LibrarySelection"]),-1),dtype=dt,chunks =(2000,),compression="gzip", compression_opts=5)
    h5_gse.create_dataset('/meta/samples/library_source',data=np.reshape(np.array(meta_df["LibrarySource"]),-1),dtype=dt,chunks =(2000,),compression="gzip", compression_opts=5)
    h5_gse.create_dataset('/meta/samples/library_strategy',data=np.reshape(np.array(meta_df["LibraryStrategy"]),-1),dtype=dt,chunks =(2000,),compression="gzip", compression_opts=5)
    h5_gse.create_dataset('/meta/samples/organism_ch1',data=np.reshape(np.array(meta_df["ScientificName"]),-1),dtype=dt,chunks =(2000,),compression="gzip", compression_opts=5)
    h5_gse.create_dataset('/meta/samples/series_id',data=np.reshape(np.array(meta_df[geo_gse_name]),-1),dtype=dt,chunks =(2000,),compression="gzip", compression_opts=5)
    h5_gse.create_dataset('/meta/samples/status',data=np.reshape(np.array(meta_df["Consent"]),-1),dtype=dt,chunks =(2000,),compression="gzip", compression_opts=5)
    h5_gse.create_dataset('/meta/samples/submission_date',data=np.reshape(np.array(meta_df["ReleaseDate"]),-1),dtype=dt,chunks =(2000,),compression="gzip", compression_opts=5)
    h5_gse.create_dataset('/meta/samples/quality',data=np.reshape(np.array(meta_df["QC_summary"]),-1),dtype=dt,chunks =(2000,),compression="gzip", compression_opts=5)
    h5_gse.create_dataset('/meta/samples/title',data=np.reshape(np.array(meta_df["BioSample"]),-1),dtype=dt,chunks =(2000,),compression="gzip", compression_opts=5)
    h5_gse.create_dataset('/meta/samples/type',data=np.full(len(meta_df["BioSample"]),"SRA", dtype=dt),chunks =(2000,),compression="gzip", compression_opts=5)
    exp_data=h5_gse.create_dataset("/data/expression", (n_genes,n_gsm),dtype= 'i4',chunks =(200,200), compression="gzip", compression_opts=7)
    srr_per_time=1000
    iter_data = pd.read_csv(filepath_or_buffer=data_file_name,sep="\t", iterator=True, chunksize=srr_per_time*n_genes,names=["srr","gene","se"])
    proc_srr= 0
    gsm_from_srr_matrix=csr_matrix(gsm_from_srr_matrix)
    print("start process expression...")
    for chunk in iter_data:
        global_ids =[]
        local_ids=[]
        local_srr_ind= {srr:ind for ind,srr in enumerate(pd.unique(chunk['srr']))}
        srr_count=len(local_srr_ind)
        proc_srr=proc_srr+srr_count
        for cur_srr in local_srr_ind:
            try:
                srr_id= srr_ind[cur_srr]
                global_ids.append(srr_id)
                local_ids.append(local_srr_ind[cur_srr])
            except(KeyError):
                continue
        if local_ids:
            print("#")
            raw_matrix= chunk["se"].values.astype(int).reshape(srr_count,n_genes)
            A=gsm_from_srr_matrix[:,global_ids] #size = all_gsm x needed_srr
            gsm_mask= np.unique(A.nonzero()[0]) # find needed gsm (rows in matrix above)
            A = A.tocsr() 
            B=csr_matrix(raw_matrix[local_ids,])
            A = A.dot(B)
            A=A.transpose()
            exp_data[0:n_genes,gsm_mask]+= A[0:n_genes,gsm_mask]            
        if proc_srr%500 ==0:
            print(proc_srr/true_srr)       
    h5_gse.flush()
print(1)

make gene index...
gene quantity: 4497
read meta file...
real SRR quantity: 8632
SRRs with gse: 4571
make SRR and gsm indexes...
GSMs quantity:
start grouping...
groupped
reordered
start h5...
create file and write meta...
start process expression...
#
0.11584800741427248
#
0.23169601482854496
#
0.34754402224281744
#
0.4633920296570899
#
0.5792400370713624
#
0.6950880444856349
#
0.8109360518999074
#
0.9267840593141798
#
1
Wall time: 1min 1s


### DEE2 STAR counts/pandas

Same as above, but more with pandas

In [1]:
%%time
import datetime
import bz2
import h5py
import numpy as np
import pandas as pd
import scipy.sparse as sps
from scipy.sparse import isspmatrix_csr,csr_matrix,csc_matrix,issparse,isspmatrix_csr, lil_matrix
import math
cache_dir="G://datasets/DEE2/"
#cache_dir="./"
organism_name= 'rnorvegicus'#'hsapiens' 'ecoli' 'mmusculus' 'athaliana' 'drerio' 'scerevisiae'celegans dmelanogaster  rnorvegicus
metadata_date = "20211102"
metadata_file = cache_dir+"metadata/"+ organism_name+"_metadata_"+metadata_date+".tsv"
source_date = "20211103"
source_name= organism_name+"_se_"+source_date+".tsv.bz2"
data_file_name= cache_dir+"raw_files/star_counts/"+source_name #celegans_se_20200530.tsv.bz2 dmelanogaster_se_20200530.tsv.bz2 ecoli_se_20200530.tsv.bz2
#drerio_se_20200530.tsv.bz2
#hsapiens_se_20200528.tsv.bz2
h5_gsm_name=cache_dir+"star_h5/"+organism_name+ "_star_matrix_"+source_date+".h5"
#h5_gsm_name = "/scratch2/mkleverov/dee2/star_h5/"+organism_name+ "_star_matrix.h5"
gene_info_file_name =cache_dir+"GeneInfo/"+organism_name+"_GeneInfo.tsv"
dt = h5py.special_dtype(vlen=np.str)
now = datetime.datetime.now()
print("make gene index...")
gene_info=pd.read_csv(filepath_or_buffer=gene_info_file_name,sep="\t")
gene_ind= {gene:ind for ind,gene in enumerate(gene_info["GeneID"])}
n_genes=len(gene_ind)
print("gene quantity: "+str(n_genes))
print("read meta file...")
iter_meta = pd.read_csv(filepath_or_buffer=metadata_file,sep="\t", iterator=True, chunksize=1000)
meta_df = pd.concat([chunk for chunk in iter_meta])
geo_acc_name = meta_df.columns[5]#'GSE_accession' # may be "GSE_accession" or "Sample_name"
geo_gse_name = meta_df.columns[6]#'Sample_name' # may be "GEO_series" "SampleName" or "Sample_name"(ecoli)
meta_df[geo_gse_name]= meta_df[geo_gse_name].astype(str)
true_srr=len(meta_df)
print("real SRR quantity: " +str(true_srr))
meta_df=meta_df[meta_df[geo_acc_name].str.startswith('GSM')]
n_srr= len(meta_df)
print("SRRs with gse: "+str(n_srr))
print("make SRR and gsm indexes...")
srr_to_gsm= dict(zip(meta_df["SRR_accession"],meta_df[geo_acc_name]))
srr_ind= {srr:ind for ind,srr in enumerate(srr_to_gsm.keys())}
gsm_ind= {gsm:ind for ind,gsm in enumerate(pd.unique(meta_df[geo_acc_name]))}
col_list = [geo_acc_name,"SRR_accession","Model","LoadDate","LibrarySelection","LibrarySource","LibraryStrategy","ScientificName",geo_gse_name,"Consent","ReleaseDate","QC_summary","BioSample"]
meta_df=meta_df[col_list]
n_gsm= len(gsm_ind)
print("GSMs quantity:" + str(n_gsm) )
gsm_from_srr_matrix= sps.lil_matrix((n_gsm,n_srr))
for srr_key in srr_to_gsm:
    gsm_from_srr_matrix[gsm_ind[srr_to_gsm[srr_key]],srr_ind[srr_key]]=1
print("start h5...")
with h5py.File(h5_gsm_name, 'w') as h5_gse:
    print("create file and write info...")
    h5_gse.create_dataset('/meta/info/version', data="10")
    h5_gse.create_dataset('/meta/info/creation_date', data=now.strftime("%Y-%m-%d"))
    h5_gse.create_dataset('/meta/info/database', data="dee2")
    h5_gse.create_dataset('/meta/info/source_file', data=source_name)
    h5_gse.create_dataset('/meta/info/author', data="Maksim Kleverov")
    h5_gse.create_dataset('/meta/info/contact', data="klevermx@gmail.com")
    exp_data=h5_gse.create_dataset("/data/expression", (n_genes,n_gsm),dtype= 'i4',chunks =(200,200), compression="gzip", compression_opts=7)
    srr_per_time=200
    iter_data = pd.read_csv(filepath_or_buffer=data_file_name,sep="\t", iterator=True, chunksize=srr_per_time*n_genes,names=["srr","gene","se"])
    proc_srr= 0
    gsm_from_srr_matrix=csr_matrix(gsm_from_srr_matrix)
    print("start process expression...")
    bad_srrs = []
    for chunk in iter_data:
        count_genes = chunk["gene"].groupby(chunk['srr'],as_index= True).nunique()
        srr_count = len(count_genes)
        proc_srr = proc_srr + srr_count       
        count_genes = count_genes[count_genes.index.isin(srr_ind)]
        local_srr_ind = count_genes[count_genes == n_genes].index
        diff = count_genes.index.difference(local_srr_ind)
        bad_srrs = [*bad_srrs, *diff]
        if len(local_srr_ind):
            local_srr_ind = pd.DataFrame(data=range(0, len(local_srr_ind)), index = local_srr_ind, columns = ["local_pos"])
            chunk = chunk[chunk["srr"].isin(local_srr_ind.index)]
            local_srr_ind['global_pos'] = local_srr_ind.index.map(srr_ind)
            #row_nums = [local_srr_ind["local_pos"][srr] for srr in chunk["srr"]]
            #col_nums = [gene_ind[gene] for gene in chunk["gene"]]
            #raw_matrix = csr_matrix((chunk["se"], (row_nums,col_nums)))
            raw_matrix = csr_matrix(chunk["se"].values.astype(int).reshape(len(local_srr_ind),n_genes))
            A=gsm_from_srr_matrix[:,local_srr_ind["global_pos"]] #size = all_gsm x needed_srr
            gsm_mask= np.unique(A.nonzero()[0]) # find needed gsm (rows in matrix above)
            A = A.tocsr() 
            A = A.dot(raw_matrix)
            A=A.transpose()
            exp_data[:,gsm_mask]+= A[:,gsm_mask] 
        if proc_srr%500 ==0:
            print(proc_srr/true_srr)
    print("publish meta...")
    print("start grouping...")

    for srr in bad_srrs:
        for col in col_list:
            if (col not in [geo_acc_name, geo_gse_name]):
                meta_df.loc[meta_df["SRR_accession"]==srr,col] = ""
    meta_df =meta_df.groupby(geo_acc_name).agg([lambda x: ';'.join(x)])
    print("groupped")
    meta_df=meta_df.loc[gsm_ind.keys()]
    print("reordered")
    gene_info=pd.read_csv(filepath_or_buffer=gene_info_file_name,sep="\t",index_col ="GeneID")
    dt = h5py.special_dtype(vlen=np.str)
    ensem = np.array(list(gene_ind.keys()),dtype=dt)
    gene_symbol = gene_info.loc[ensem]["GeneSymbol"]
    h5_gse.create_dataset('/meta/genes/ensembl_gene_id',data=ensem,chunks =(len(gene_ind),),compression="gzip", compression_opts=7)
    h5_gse.create_dataset('/meta/genes/gene_symbol',data=np.array(gene_symbol, dtype =dt),chunks =(len(gene_ind),),compression="gzip", compression_opts=7)
    h5_gse.create_dataset('/meta/info/description', data="Contain all SRRs from dee2 for which sample_geo_accession starts with gse. Star estimates was groupped by sample_geo_accession and summed up")
    h5_gse.create_dataset('/meta/genes/genes',data=np.array(gene_symbol, dtype =dt),chunks =(len(gene_ind),),compression="gzip", compression_opts=7)
    h5_gse.create_dataset('/meta/samples/geo_accession',data=np.array(list(gsm_ind.keys()), dtype =dt),chunks =(2000,),compression="gzip", compression_opts=5)
    h5_gse.create_dataset('/meta/samples/SRR_accession',data= np.reshape(np.array(meta_df["SRR_accession"]),-1),dtype=dt,chunks =(2000,),compression="gzip", compression_opts=5)
    h5_gse.create_dataset('/meta/samples/instrument_model',data=np.reshape(np.array(meta_df["Model"]),-1),dtype=dt,chunks =(2000,),compression="gzip", compression_opts=5)
    h5_gse.create_dataset('/meta/samples/last_update_date',data=np.reshape(np.array(meta_df["LoadDate"]),-1),dtype=dt,chunks =(2000,),compression="gzip", compression_opts=5)
    h5_gse.create_dataset('/meta/samples/library_selection',data=np.reshape(np.array(meta_df["LibrarySelection"]),-1),dtype=dt,chunks =(2000,),compression="gzip", compression_opts=5)
    h5_gse.create_dataset('/meta/samples/library_source',data=np.reshape(np.array(meta_df["LibrarySource"]),-1),dtype=dt,chunks =(2000,),compression="gzip", compression_opts=5)
    h5_gse.create_dataset('/meta/samples/library_strategy',data=np.reshape(np.array(meta_df["LibraryStrategy"]),-1),dtype=dt,chunks =(2000,),compression="gzip", compression_opts=5)
    h5_gse.create_dataset('/meta/samples/organism_ch1',data=np.reshape(np.array(meta_df["ScientificName"]),-1),dtype=dt,chunks =(2000,),compression="gzip", compression_opts=5)
    h5_gse.create_dataset('/meta/samples/series_id',data=np.reshape(np.array(meta_df[geo_gse_name]),-1),dtype=dt,chunks =(2000,),compression="gzip", compression_opts=5)
    h5_gse.create_dataset('/meta/samples/status',data=np.reshape(np.array(meta_df["Consent"]),-1),dtype=dt,chunks =(2000,),compression="gzip", compression_opts=5)
    h5_gse.create_dataset('/meta/samples/submission_date',data=np.reshape(np.array(meta_df["ReleaseDate"]),-1),dtype=dt,chunks =(2000,),compression="gzip", compression_opts=5)
    h5_gse.create_dataset('/meta/samples/quality',data=np.reshape(np.array(meta_df["QC_summary"]),-1),dtype=dt,chunks =(2000,),compression="gzip", compression_opts=5)
    h5_gse.create_dataset('/meta/samples/title',data=np.reshape(np.array(meta_df["BioSample"]),-1),dtype=dt,chunks =(2000,),compression="gzip", compression_opts=5)
    h5_gse.create_dataset('/meta/samples/type',data=np.full(len(meta_df["BioSample"]),"SRA", dtype=dt),chunks =(2000,),compression="gzip", compression_opts=5)
    
    h5_gse.flush()
print(1)

make gene index...
gene quantity: 32883
read meta file...
real SRR quantity: 28753
SRRs with gse: 19677
make SRR and gsm indexes...
GSMs quantity:15773
start h5...
create file and write info...
start process expression...
0.03477897958473898
0.06955795916947796
0.10433693875421696
0.13911591833895592
0.17389489792369492
0.2086738775084339
0.24345285709317288
0.27823183667791185
0.31301081626265087
0.34778979584738984
0.3825687754321288
0.4173477550168678
0.4521267346016068
0.48690571418634576
0.5216846937710847
0.5564636733558237
0.5912426529405628
0.6260216325253017
0.6608006121100407
0.6955795916947797
0.7303585712795186
0.7651375508642576
0.7999165304489966
0.8346955100337357
0.8694744896184746
0.9042534692032136
0.9390324487879526
0.9738114283726915
publish meta...
start grouping...
groupped
reordered
1
Wall time: 1h 7min 48s


In [82]:
%%time
import datetime
import bz2
import h5py
import numpy as np
import pandas as pd
import scipy.sparse as sps
from scipy.sparse import isspmatrix_csr,csr_matrix,csc_matrix,issparse,isspmatrix_csr, lil_matrix
import math
cache_dir="G://datasets/DEE2/"
#cache_dir="./"
organism_name= 'hsapiens'#'hsapiens' 'ecoli' 'mmusculus' 'athaliana' 'drerio' 'scerevisiae'celegans dmelanogaster  rnorvegicus
metadata_date = "20210319"
metadata_file = cache_dir+"metadata/"+ organism_name+"_metadata_"+metadata_date+".tsv"
source_date = "20210319"
source_name= organism_name+"_se_"+source_date+".tsv.bz2"
data_file_name= cache_dir+"raw_files/star_counts/"+source_name #celegans_se_20200530.tsv.bz2 dmelanogaster_se_20200530.tsv.bz2 ecoli_se_20200530.tsv.bz2
#drerio_se_20200530.tsv.bz2
#hsapiens_se_20200528.tsv.bz2
h5_gsm_name=cache_dir+"star_h5/"+organism_name+ "_star_matrix_"+source_date+".h5"
#h5_gsm_name = "/scratch2/mkleverov/dee2/star_h5/"+organism_name+ "_star_matrix.h5"
gene_info_file_name =cache_dir+"GeneInfo/"+organism_name+"_GeneInfo.tsv"
dt = h5py.special_dtype(vlen=np.str)
now = datetime.datetime.now()
print("make gene index...")
gene_info=pd.read_csv(filepath_or_buffer=gene_info_file_name,sep="\t")
gene_ind= {gene:ind for ind,gene in enumerate(gene_info["GeneID"])}
n_genes=len(gene_ind)
print("gene quantity: "+str(n_genes))
print("read meta file...")
iter_meta = pd.read_csv(filepath_or_buffer=metadata_file,sep="\t", iterator=True, chunksize=1000)
meta_df = pd.concat([chunk for chunk in iter_meta])
geo_acc_name = meta_df.columns[5]#'GSE_accession' # may be "GSE_accession" or "Sample_name"
geo_gse_name = meta_df.columns[6]#'Sample_name' # may be "GEO_series" "SampleName" or "Sample_name"(ecoli)


make gene index...
gene quantity: 58302
read meta file...
Wall time: 16 s


In [84]:
meta_df

,SRR_accession,QC_summary,SRX_accession,SRS_accession,SRP_accession,Sample_name,GEO_series,Library_name,SampleName,ReleaseDate,...,Affection_Status,Analyte_Type,Histological_Type,Body_Site,CenterName,Submission,dbgap_study_accession,Consent,RunHash,ReadHash
0,DRR000003,"FAIL(4,5,6,7)",DRX000003,DRS000003,DRP000003,SAMD00013986,NaN,DLD1_normoxia_nucleosome,SAMD00013986,2010-10-14 00:53:29,...,NaN,NaN,NaN,NaN,UT-MGS,DRA000003,NaN,public,57A45899FA02E1A183A0ACA9F65E6A0A,D602C0CD8F2B562BA085FB8FC49C964B
1,DRR000004,"FAIL(4,5,6,7)",DRX000003,DRS000003,DRP000003,SAMD00013986,NaN,DLD1_normoxia_nucleosome,SAMD00013986,2010-10-14 00:53:29,...,NaN,NaN,NaN,NaN,UT-MGS,DRA000003,NaN,public,D246572C1CAADBF3F601A23B6F7CD289,6FC67E696F9658B211BD15844D89C4AC
2,DRR000005,"FAIL(4,5,6,7)",DRX000003,DRS000003,DRP000003,SAMD00013986,NaN,DLD1_normoxia_nucleosome,SAMD00013986,2010-10-14 00:53:29,...,NaN,NaN,NaN,NaN,UT-MGS,DRA000003,NaN,public,92FC08D16F287F47B6AE70F92CD0AB4C,025670D28DB6462F558A4C31DC4D646B
3,DRR000006,"FAIL(4,5,6,7)",DRX000003,DRS000003,DRP000003,SAMD00013986,NaN,DLD1_normoxia_nucleosome,SAMD00013986,2010-10-14 00:53:29,...,NaN,NaN,NaN,NaN,UT-MGS,DRA000003,NaN,public,E9B011701C2E7B972FAF33CD29840F53,D161FC28BB386893EA52829DFF3D16C7
4,DRR000007,"FAIL(4,5,6,7)",DRX000003,DRS000003,DRP000003,SAMD00013986,NaN,DLD1_normoxia_nucleosome,SAMD00013986,2010-10-14 00:53:29,...,NaN,NaN,NaN,NaN,UT-MGS,DRA000003,NaN,public,850599A62FFA87DF2E39281FC127C397,BDD124DC6EFF61D7A50A9CA6F8A8F6EB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
604971,SRR9995485,"WARN(1,5,7)",SRX6734453,SRS5285729,SRP218732,GSM4038638,NaN,NaN,GSM4038638,2020-06-12 16:15:18,...,NaN,NaN,NaN,NaN,GEO,SRA944214,NaN,public,1BAD74B91D3650838330DF8834ECFF93,E23A78A8A16CCF9D3C76BE63D0FAB6B0
604972,SRR9995486,"WARN(1,5,7)",SRX6734453,SRS5285729,SRP218732,GSM4038638,NaN,NaN,GSM4038638,2020-06-12 16:15:18,...,NaN,NaN,NaN,NaN,GEO,SRA944214,NaN,public,601BBF0AEA492785E30D9997022B628C,E58BD173F8E2A8A88D9D0B9BE45666AD
604973,SRR9995490,"WARN(1,5,7)",SRX6734455,SRS5285731,SRP218732,GSM4038640,NaN,NaN,GSM4038640,2020-06-12 16:15:18,...,NaN,NaN,NaN,NaN,GEO,SRA944214,NaN,public,70753910DD0C02C64FD6541AC2028D0F,44662CBEBBC66B6DFBB06A20B74685AD
604974,SRR9995491,"WARN(1,5,7)",SRX6734455,SRS5285731,SRP218732,GSM4038640,NaN,NaN,GSM4038640,2020-06-12 16:15:18,...,NaN,NaN,NaN,NaN,GEO,SRA944214,NaN,public,13BEC07E0A6B45D76FF4F181590E252B,E625B95FE9A20A7CB4742F8C671F3C4A


In [72]:
meta_df

,Sample_name,SRR_accession,Model,LoadDate,LibrarySelection,LibrarySource,LibraryStrategy,ScientificName,GEO_series,Consent,ReleaseDate,QC_summary,BioSample
126715,GSM402329,SRR029054,Illumina Genome Analyzer,2014-05-26 09:27:28,other,TRANSCRIPTOMIC,OTHER,Homo sapiens,nan,public,2009-10-21 17:33:21,"FAIL(3,4,5,6,7)",SAMN00004353
126716,GSM416753,SRR029124,Illumina Genome Analyzer,2014-05-26 09:26:27,size fractionation,TRANSCRIPTOMIC,RNA-Seq,Homo sapiens,nan,public,2009-11-03 13:20:36,"FAIL(3,4,5,6,7)",SAMN00004408
126717,GSM416754,SRR029125,Illumina Genome Analyzer,2014-05-26 09:26:29,size fractionation,TRANSCRIPTOMIC,RNA-Seq,Homo sapiens,nan,public,2009-11-03 13:20:48,"FAIL(3,4,5,6,7)",SAMN00004409
126718,GSM416755,SRR029126,Illumina Genome Analyzer,2014-05-26 09:26:31,size fractionation,TRANSCRIPTOMIC,RNA-Seq,Homo sapiens,nan,public,2009-11-03 13:20:31,"FAIL(3,4,5,6,7)",SAMN00004410
126719,GSM416756,SRR029127,Illumina Genome Analyzer,2014-05-26 09:26:35,size fractionation,TRANSCRIPTOMIC,RNA-Seq,Homo sapiens,nan,public,2009-11-03 13:20:57,"FAIL(3,4,5,6,7)",SAMN00004411
...,...,...,...,...,...,...,...,...,...,...,...,...,...
604971,GSM4038638,SRR9995485,Illumina HiSeq 2500,2019-08-19 12:14:08,cDNA,TRANSCRIPTOMIC,RNA-Seq,Homo sapiens,nan,public,2020-06-12 16:15:18,"WARN(1,5,7)",SAMN12600296
604972,GSM4038638,SRR9995486,Illumina HiSeq 2500,2019-08-19 12:16:54,cDNA,TRANSCRIPTOMIC,RNA-Seq,Homo sapiens,nan,public,2020-06-12 16:15:18,"WARN(1,5,7)",SAMN12600296
604973,GSM4038640,SRR9995490,Illumina HiSeq 2500,2019-08-19 12:17:36,cDNA,TRANSCRIPTOMIC,RNA-Seq,Homo sapiens,nan,public,2020-06-12 16:15:18,"WARN(1,5,7)",SAMN12600294
604974,GSM4038640,SRR9995491,Illumina HiSeq 2500,2019-08-19 12:15:20,cDNA,TRANSCRIPTOMIC,RNA-Seq,Homo sapiens,nan,public,2020-06-12 16:15:18,"WARN(1,5,7)",SAMN12600294


In [74]:
bad_srrs = ['SRR029054','SRR9995486']
for srr in bad_srrs:
        ind = meta_df["SRR_accession"]==srr
        for col in col_list:
            if (col not in [geo_acc_name, geo_gse_name]):     
                meta_df.loc[ind,col] = ""


In [22]:
bad_srrs

[]

In [76]:
meta_df.loc[meta_df["SRR_accession"]=="SRR029054"]

,Sample_name,SRR_accession,Model,LoadDate,LibrarySelection,LibrarySource,LibraryStrategy,ScientificName,GEO_series,Consent,ReleaseDate,QC_summary,BioSample


In [51]:
meta_df.loc[meta_df["SRR_accession"]=="SRR029124",'LoadDate'] = ''

In [54]:
meta_df.loc[meta_df["SRR_accession"]=="SRR029124", ["LibrarySelection", "ScientificName"] ] =""

In [55]:
meta_df.loc[meta_df["SRR_accession"]=="SRR029124"]

,Sample_name,SRR_accession,Model,LoadDate,LibrarySelection,LibrarySource,LibraryStrategy,ScientificName,GEO_series,Consent,ReleaseDate,QC_summary,BioSample
126716,GSM416753,SRR029124,Illumina Genome Analyzer,,,TRANSCRIPTOMIC,RNA-Seq,,nan,public,2009-11-03 13:20:36,"FAIL(3,4,5,6,7)",SAMN00004408


In [56]:
meta_df.loc[meta_df["SRR_accession"]=="SRR029124", "LibrarySelection"]

126716    
Name: LibrarySelection, dtype: object

In [11]:
indices

[0, 1, 0, 2, 3, 1]

In [21]:
csr_matrix((data, indices, indptr), dtype=int).toarray()

array([[2, 1, 0, 0],
       [0, 1, 1, 1]])

In [18]:
indices[indptr[1]:indptr[2]]

[2, 3, 1]

SRR/h5 file with matrices Работает так же долго

In [40]:
%%time
import datetime
from scipy.sparse import isspmatrix_csr,csr_matrix,csc_matrix
data_file_name= cache_dir+"raw_files/"+organism_name+"_ke.tsv.bz2"
#data_file_name="F://mmusculus_ke.tsv.bz2"
h5_gsm_name = cache_dir+organism_name+ "_kalisto_SRR_matrix.h5"
now = datetime.datetime.now()
print("start h5")
with h5py.File(h5_gsm_name, 'w') as h5_gse:
    meta_grp = h5_gse.create_group('meta')
    info_grp = h5_gse.create_group('info')
    data_grp= h5_gse.create_group('data')
    info_grp.create_dataset('version', data="dee2_gse_v1")
    info_grp.create_dataset('creation_date', data=now.strftime("%Y.%m.%d"))
    n_genes =len(gene_ind)
    dt = h5py.special_dtype(vlen=np.str) 
    meta_grp.create_dataset('genes',data=srr_to_gsm.values(),dtype=dt)
    meta_grp.create_dataset('SRR',data=srr_ind.keys(),dtype=dt)
    meta_grp.create_dataset('Sample_geo_accession',data=meta_df[geo_acc_name], dtype =dt)
    meta_grp.create_dataset('Sample_instrument_model',(n_srr,), dtype =dt)
    meta_grp.create_dataset('Sample_last_update_date',(n_srr,), dtype =dt)
    meta_grp.create_dataset('Sample_library_selection',(n_srr,), dtype =dt)
    meta_grp.create_dataset('Sample_library_source',(n_srr,), dtype =dt)
    meta_grp.create_dataset('Sample_library_strategy',(n_srr,), dtype =dt)
    meta_grp.create_dataset('Sample_organism_ch1',(n_srr,),dtype =dt)
    meta_grp.create_dataset('Sample_series_id',(n_srr,), dtype =dt)
    meta_grp.create_dataset('Sample_status',(n_srr,), dtype =dt)
    meta_grp.create_dataset('Sample_submission_date',(n_srr,), dtype =dt)
    meta_grp.create_dataset('Sample_taxid_ch1',(n_srr,), dtype =dt)
    meta_grp.create_dataset('Sample_quality',(n_srr,), dtype =dt)
    meta_grp.create_dataset('Sample_title',(n_srr,), dtype =dt)
    meta_grp.create_dataset('Sample_type',data=np.full(n_srr,"SRA", dtype =dt))
    exp_data=data_grp.create_dataset("expression", (n_srr,n_genes),dtype= 'i4')#, compression="gzip", compression_opts=9)
    srr_per_time=1000
    row_length= len(tx_ind)
    print(row_length)
    iter_data = pd.read_csv(filepath_or_buffer=data_file_name,sep="\t", iterator=True, chunksize=srr_per_time*row_length,names=["srr","tx","ke"])
    proc_srr= 0
    print("start cast matrices")  
    tx_to_gene_matrix=csr_matrix(tx_to_gene_matrix)
    print("start process expression")
    for chunk in iter_data:
        global_ids =[]
        local_ids=[]
        local_srr_ind= {srr:ind for ind,srr in enumerate(pd.unique(chunk['srr']))}
        srr_count=len(local_srr_ind)
        proc_srr=proc_srr+srr_count
        for cur_srr in sorted(local_srr_ind.keys()):
            try:
                srr_id= srr_ind[cur_srr]
                global_ids.append(srr_id)
                local_ids.append(local_srr_ind[cur_srr])
            except(KeyError):
                continue
        
        if local_ids:           
            raw_matrix= chunk["ke"].values.astype(int).reshape(srr_count,row_length)
            A=csr_matrix(raw_matrix[local_ids,:])
            A = A.dot(tx_to_gene_matrix)
            exp_data[global_ids,0:n_genes]= exp_data[global_ids,0:n_genes]+A[:,0:n_genes]
        if proc_srr%500 ==0:
            print(proc_srr/true_srr)
    h5_gse.flush()
print(1)


start h5
4322
start cast matrices
start process expression
0.14236902050113895
0.2847380410022779
0.4271070615034169
0.5694760820045558
0.7118451025056948
0.8542141230068337
0.9965831435079726
1
Wall time: 29 s


dee2 kalisto to h5 files